In [1]:
# For the URL for Sending a POST request to the Applet created for this purpose

# https://ifttt.com/maker_webhooks/settings

# URL given in the above link: https://maker.ifttt.com/use/olYyWzU82vyL3qxlOZB9YUHKXyzXm8-au1rKhsGUNXQ

#'https://maker.ifttt.com/trigger/news_event/with/key/olYyWzU82vyL3qxlOZB9YUHKXyzXm8-au1rKhsGUNXQ'

#https://github.com/dbader/schedule/issues/209

In [ ]:
# Here we again use ifttt to deliver suggested articles to personal emailids
# For Making Applet, Use if this then that - for "if", use Maker (in that choose "recieve a web request" and type "news_event" and click crete trigger) (recieves POST request)and "then" use "Gmail" (in that select send an email) (have to authorize first, i.e. allow the app to access Gmail - just follow instuctions to authorize)
# For Gmail, u can then customize the body and message - added mom's email address, {Value1} which represents title and {Value2} which represents URL of thee news article
# craete a function and schedule it to run for every 60 minutes.
# In that fucntion, (first steps are same - download using gspread, use beautiful soup and predict tag) if u get a new news artcile in the spreadsheet, then we embed the title and url of it in a json string called Value1 and make a post request (that includes the data) to this new applet created in the ifttt ('https://maker.ifttt.com/trigger/news_event/with/key/olYyWzU82vyL3qxlOZB9YUHKXyzXm8-au1rKhsGUNXQ') - key is from got on making the applet
# this causes the app to send an email to the inbox
# Clean up the worksheet after this tto get new articles every time


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
import schedule
import time
import pickle
import json
import gspread
import requests
from oauth2client.service_account import ServiceAccountCredentials
from bs4 import BeautifulSoup

pd.set_option('display.max_colwidth',250)

def fetch_news():
    try:
        vect = pickle.load(open(r'/Users/rishitdholakia/Desktop/Projects/Cooking News Feed/vect_pickle.p','rb'))
        model = pickle.load(open(r'/Users/rishitdholakia/Desktop/Projects/Cooking News Feed/model_pickle.p','rb'))
    
        scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
        credentials=ServiceAccountCredentials.from_json_keyfile_name(r'/Users/rishitdholakia/Desktop/Projects/Cooking News Feed/riya-latest-project-0add4349491b.json',scopes=scope)
        gc=gspread.authorize(credentials)
        
        ws = gc.open("CookingArticles")
        # Or sheet = gc.open('CookingArticles').sheet1
        sh=ws.sheet1
        zd=list(zip(sh.col_values(2),sh.col_values(3),sh.col_values(4)))
        zf=pd.DataFrame(zd,columns=['title','urls','html'])
        zf.replace('',pd.np.nan,inplace=True)
        zf.dropna(inplace=True)
                            
        def get_text(x):
            soup=BeautifulSoup(x,'lxml')
            text=soup.get_text()
            return text
                            
        df.loc[:,'text']=df['html'].map(get_text)
                            
        tv=vect.transform(zf['text'])
        res=model.predict(tv)
                            
        rf=pd.DataFrame(res,columns=['wanted'])
        rez=pd.merge(rf,zf,left_index=True,right_index=True)
                            
        news_str=''
        for t,u in zip(rez[rez['wanted']=='y']['title'],rez[rez['wanted']=='y']['urls']):
            news_str=news_str + t +'\n'+ u + '\n'
                            
        payload = {"value1" : news_str}
        r = requests.post('https://maker.ifttt.com/trigger/news_event/with/key/olYyWzU82vyL3qxlOZB9YUHKXyzXm8-au1rKhsGUNXQ',data=payload)
                            
        #Clean Up Worksheet
        lenv = len(sh.col_values(1))
        cell_list=sh.range('A1:F' + str(lenv))
        for cell in cell_list:
            cell.value=''
        sh.update_cells(cell_list)
                            
        print(r.text)
    
    except:
        print('Failed')
    
schedule.every(5).minutes.do(fetch_news)
                            
while 1:
    schedule.run_pending()
    time.sleep(1)
                            